# 14-Seq2Seq, Attention

[1.Define Vocab](#1)

[2.Data Preprocessing](#2)

[3.Load & Read Dataset](#3)

[4.Sequence to Sequence](#4)
 - [4-1.Encoder](#4-1)
 - [4-2.Decoder](#4-2)
 - [4-3.Training](#4-3)
 - [4-4.Evaluate](#4-4)

[5.Attention Decoder](#5)
 - [5-1.Attention with Decoder](#5-1)
 - [5-2.Training](#5-2)
 - [5-3.Evaluate](#5-3)

In [1]:
import re
import random
import numpy as np
import unicodedata
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import torch
import torch.nn as nn
import torch.nn.functional as F


from io import open
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

<a id="1"></a>
## 1.Define Vocab

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MAX_LENGTH = 10
SOS_TOKEN = 0
EOS_TOKEN = 1
TEACHER_FORCING_RATIO = 0.5
HIDDEN_DIM = 256

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

<a id="2"></a>
## 2.Data Preprocessing

In [3]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    
    return s.strip()

In [4]:
eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    ## MAX_LENGTH를 초과하는지, eng_prefixes에 해당하는 것으로 영문장이 시작되는지 검사. True or False를 반환.
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH and p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    ## True를 반환 받으면 리스트에 추가하고, False를 받으면 리스트에 추가하지 않는다.
    return [pair for pair in pairs if filterPair(pair)]

<a id="3"></a>
## 3.Load & Read Dataset

In [5]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open(f'/home/pervinco/Datasets/en-fr/data/{lang1}-{lang2}.txt', encoding='utf-8').read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines] ## Unicode -> Ascii + (fr_sentence, en_sentence)

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [6]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 11445 sentence pairs
Counting words...
Counted words:
fra 4601
eng 2991
['il ne se trouve pas en ville', 'he s out of town']


<a id="4"></a>
## 4.Sequence to Sequence

<a id="4-1"></a>
### 4-1.Encoder

In [7]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.hidden_dim = hidden_dim

        self.embedding = nn.Embedding(input_dim, hidden_dim)
        self.lstm = nn.LSTM(hidden_dim, hidden_dim)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, (hidden, cell) = self.lstm(output, hidden)  # LSTM은 hidden과 cell state를 반환
        return output, (hidden, cell)

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_dim, device=device),  # hidden state 초기화
                torch.zeros(1, 1, self.hidden_dim, device=device))  # cell state 초기화

<a id="4-2"></a>
### 4-2.Decoder

In [8]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, (hidden, cell) = self.lstm(output, hidden)  # LSTM 사용으로 hidden과 cell 상태 업데이트
        output = self.softmax(self.out(output[0]))  # Softmax 활성화 함수 사용
        return output, (hidden, cell)

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),  # hidden state 초기화
                torch.zeros(1, 1, self.hidden_size, device=device))  # cell state 초기화

<a id="4-3"></a>
### 4-3.Training

In [9]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    loss = 0

    # Encoder
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        # LSTM의 encoder_hidden은 (hidden state, cell state) 튜플

    # Decoder
    decoder_input = torch.tensor([[SOS_TOKEN]], device=device)
    decoder_hidden = encoder_hidden  # Encoder의 마지막 hidden state를 Decoder의 초기 hidden state로 사용

    use_teacher_forcing = True if random.random() < TEACHER_FORCING_RATIO else False

    if use_teacher_forcing:
        # Teacher forcing: 다음 입력으로 목표(target) 사용
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di].unsqueeze(0)  # Teacher forcing 사용

    else:
        # Teacher forcing 없음: 자신의 예측을 다음 입력으로 사용
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1) ## 디코더가 출력한 결과 중에서 가장 높은 확률을 가진 원소 하나를 선택
            decoder_input = topi.squeeze().detach().unsqueeze(0)  # 입력을 위해 squeeze 후 detach

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_TOKEN:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [10]:
def indexesFromSentence(lang, sentence):
    """
    주어진 문장을 공백을 기준으로 단어로 분리한 뒤, 각 단어에 해당하는 인덱스를 lang의 word2index 딕셔너리를 사용하여 찾아낸다. 
    결과적으로 문장을 인덱스의 배열로 변환.
    """
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    """
    문장을 인덱스 배열로 변환한 후, 특별한 토큰인 EOS_token (End Of Sentence 토큰, 문장의 끝을 나타냄)을 배열에 추가.
    배열을 Tensor로 변환.
    """
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_TOKEN)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    
    return (input_tensor, target_tensor)


def trainIters(encoder, decoder, n_iters, print_every=1000, learning_rate=0.01):
    print_loss_total = 0
    plot_loss_total = 0

    encoder_optimizer = torch.optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = torch.optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print(f"Iter : {iter}, Loss : {print_loss_avg:.4f}")

In [11]:
encoder = Encoder(input_lang.n_words, HIDDEN_DIM).to(device)
decoder = Decoder(HIDDEN_DIM, output_lang.n_words).to(device)

trainIters(encoder, decoder, 100000, print_every=5000)

Iter : 5000, Loss : 3.0969
Iter : 10000, Loss : 2.6752
Iter : 15000, Loss : 2.4425
Iter : 20000, Loss : 2.1947
Iter : 25000, Loss : 1.9700
Iter : 30000, Loss : 1.7812
Iter : 35000, Loss : 1.6168
Iter : 40000, Loss : 1.4359
Iter : 45000, Loss : 1.3210
Iter : 50000, Loss : 1.2174
Iter : 55000, Loss : 1.1165
Iter : 60000, Loss : 1.0301
Iter : 65000, Loss : 0.9261
Iter : 70000, Loss : 0.8489
Iter : 75000, Loss : 0.7715
Iter : 80000, Loss : 0.6923
Iter : 85000, Loss : 0.6457
Iter : 90000, Loss : 0.6061
Iter : 95000, Loss : 0.5434
Iter : 100000, Loss : 0.4974


<a id="4-4"></a>
### 4-4.Evaluate

In [12]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size(0)
        encoder_hidden = encoder.initHidden()

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei].unsqueeze(0), encoder_hidden)

        decoder_input = torch.tensor([[SOS_TOKEN]], device=device)  # SOS
        decoder_hidden = encoder_hidden  # LSTM의 경우 (hidden, cell) 튜플 사용

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_TOKEN:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach().unsqueeze(0)

        return decoded_words

def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')


In [13]:
evaluateRandomly(encoder, decoder)

> nous attendons la publication de son ouvrage
= we are expecting the publication of his book
< we are expecting the of his book <EOS>

> je suis trop fatigue pour penser
= i m too tired to think
< i m too tired to think <EOS>

> vous n etes pas autorise a manger ceux la
= you re not allowed to eat those
< you re not allowed to eat those <EOS>

> nous sommes jalouses
= we re jealous
< we re jealous <EOS>

> je suis tres serieuse
= i m very serious
< i m very serious <EOS>

> elle a les pieds en dedans
= she is pigeon toed
< she is walking in chinese <EOS>

> je suis tres gros
= i m very fat
< i m very fat <EOS>

> elles sont mignonnes
= they re cute
< they re cute <EOS>

> nous sommes sous son commandement
= we are under his command
< we are under his his <EOS>

> vous etes precisement l homme que je veux voir
= you are the very man i want to see
< you re the the man i want <EOS>



<a id="5-1"></a>
## 5-1.Attention with Decoder

In [14]:
class AttnDecoder(nn.Module):
    def __init__(self, hidden_dim, output_dim, dropout_p=0.1, max_length=MAX_LENGTH):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_dim, self.hidden_dim)
        self.attn = nn.Linear(self.hidden_dim * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_dim * 2, self.hidden_dim)
        self.dropout = nn.Dropout(self.dropout_p)
        self.lstm = nn.LSTM(self.hidden_dim, self.hidden_dim)  # GRU 대신 LSTM 사용
        self.out = nn.Linear(self.hidden_dim, self.output_dim)
        self.softmax = nn.LogSoftmax(dim=1)  # Softmax 대신 LogSoftmax 사용

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden[0][0]), 1)), dim=1)  # attention score를 계산하고, attention dist를 구함.
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),encoder_outputs.unsqueeze(0)) # attention value

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.lstm(output, hidden)  # LSTM 사용

        output = self.softmax(self.out(output[0]))
        return output, hidden, attn_weights

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_dim, device=device),  # hidden state 초기화
                torch.zeros(1, 1, self.hidden_dim, device=device))  # cell state 초기화


<a id="5-2"></a>
## 5-2.Training

In [15]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_dim, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei].unsqueeze(0), encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_TOKEN]], device=device)
    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < TEACHER_FORCING_RATIO else False

    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden, _ = decoder(decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])  # .unsqueeze(0) 제거
            decoder_input = target_tensor[di].unsqueeze(0)  # target_tensor[di]가 스칼라 값인 경우
    else:
        for di in range(target_length):
            decoder_output, decoder_hidden, _ = decoder(decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach().unsqueeze(0)

            loss += criterion(decoder_output, target_tensor[di])  # .unsqueeze(0) 제거
            if decoder_input.item() == EOS_TOKEN:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [16]:
encoder = Encoder(input_lang.n_words, HIDDEN_DIM).to(device)
decoder = AttnDecoder(HIDDEN_DIM, output_lang.n_words).to(device)

trainIters(encoder, decoder, 100000, print_every=5000)

Iter : 5000, Loss : 3.0957
Iter : 10000, Loss : 2.6877
Iter : 15000, Loss : 2.3743
Iter : 20000, Loss : 2.1446
Iter : 25000, Loss : 1.8948
Iter : 30000, Loss : 1.7686
Iter : 35000, Loss : 1.6334
Iter : 40000, Loss : 1.4668
Iter : 45000, Loss : 1.3585
Iter : 50000, Loss : 1.2681
Iter : 55000, Loss : 1.1454
Iter : 60000, Loss : 1.0421
Iter : 65000, Loss : 0.9802
Iter : 70000, Loss : 0.9094
Iter : 75000, Loss : 0.8256
Iter : 80000, Loss : 0.7575
Iter : 85000, Loss : 0.7183
Iter : 90000, Loss : 0.6538
Iter : 95000, Loss : 0.6049
Iter : 100000, Loss : 0.5695


<a id="5-3"></a>
### 5-3.Evaluate

In [17]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size(0)
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_dim, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei].unsqueeze(0), encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_TOKEN]], device=device)
        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden, _ = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_TOKEN:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach().unsqueeze(0)

        return decoded_words
    

def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [18]:
evaluateRandomly(encoder, decoder)

> je me rejouis de te voir heureux
= i m glad to see you re happy
< i m glad to see you re happy <EOS>

> c est un scientifique de niveau international
= he s a world class scientist
< he s a strange class <EOS>

> il revasse toujours
= he is always day dreaming
< he is always day <EOS>

> nous n y sommes pas habitues
= we re not used to it
< we re not used to it <EOS>

> nous sommes en guerre
= we are at war
< we re at war <EOS>

> tu es vraiment egoiste
= you re really selfish
< you re really selfish <EOS>

> tu es trop vieux pour moi
= you re too old for me
< you re too old for me <EOS>

> vous etes bizarres
= you re weird
< you re weird <EOS>

> il est mechant
= he is nasty
< he is lazy <EOS>

> elles sont autosuffisantes
= they re self sufficient
< they re self sufficient <EOS>

